In [1]:
import pandas as pd
import pickle

import ipywidgets as widgets
from ipywidgets import interact

from src.data.data_load import (
    load_tables, 
    load_online_instance, 
    load_distances, 
    upload_ONLINE_static_solution
)
from src.data.solution_load import load_solution_dfs
from src.utils.filtering import flexible_filter
from src.utils.plotting import plot_metrics_comparison_dynamic
from src.data.metrics import collect_results_to_df, compute_metrics_with_moves, get_day_plotting_df
from src.config.experimentation_config import *
from src.config.config import *

data_path = '../data'

instance = 'instAD3'        # Options: ['instAS1', 'instRS1']

distance_type = 'osrm'              # Options: ['osrm', 'manhattan']
dist_method = 'haversine'      # Options: ['precalced', 'haversine']

optimization_obj = 'driver_distance'

directorio_df, labors_raw_df, cities_df, duraciones_df, valid_cities = load_tables(data_path, generate_labors=False)
labors_real_df, labors_static_df, labors_dynamic_df = load_online_instance(data_path, instance, labors_raw_df)
dist_dict = load_distances(data_path, distance_type, instance, dist_method)

fechas = fechas_map(instance)

metricas = ['service_count', 'vt_count', 'num_drivers', 'driver_extra_time', 'driver_move_distance']

# Cargar resultados

In [2]:
algo_selection = {
    'hist': True,
    'offline': True,
    'INSERT': True,
    'INSERT_BUFFER': True,
    'REACT': True,
    'REACT_BUFFER': True
}

In [3]:
algorithms = []
include_all_city = True

if algo_selection['hist']:
    labors_hist_df, moves_hist_df, postponed_labors_hist = load_solution_dfs(
        data_path, 
        instance, 
        dist_method,
        algorithm='hist',
        include_all_city=include_all_city
    )
    algorithms.append({
        "name": "Histórico",
        "labors_df": labors_hist_df,
        "moves_df": moves_hist_df,
        "type": "historic",
        "color": "#C0392B",  # 🔴 Deep crimson red – “you were here”
        "visible": True,
    })

if algo_selection['offline']:    
    labors_algo_baseline_df, moves_algo_baseline_df, postponed_labors_OFFLINE = load_solution_dfs(
        data_path, 
        instance, 
        dist_method,
        algorithm='OFFLINE',
        include_all_city=include_all_city
    )
    algorithms.append({
        "name": "Offline",
        "labors_df": labors_algo_baseline_df,
        "moves_df": moves_algo_baseline_df,
        "type": "algorithm",
        "color": "#2980B9",  # 🔵 Royal blue – “best possible solution”
        "visible": True,
    })

labors_algo_static_df, moves_algo_static_df, postponed_labors_ONLINE_static = load_solution_dfs(
    data_path,
    instance,
    dist_method,
    algorithm='ONLINE_static',
    include_all_city=include_all_city
)

if algo_selection['INSERT']:
    labors_algo_INSERT_df, moves_algo_INSERT_df, postponed_labors_INSERT = load_solution_dfs(
        data_path,
        instance,
        dist_method,
        algorithm='INSERT',
        include_all_city=include_all_city
    )
    algorithms.append({
        "name": "INSERT",
        "labors_df": labors_algo_INSERT_df,
        "moves_df": moves_algo_INSERT_df,
        "type": "algorithm",
        "color": "#27AE60",  # 🟢 Vibrant green – “first smart improvement”
        "visible": True,
    })

if algo_selection['INSERT_BUFFER']:
    labors_algo_INSERT_BUFFER_df, moves_algo_INSERT_BUFFER_df, postponed_labors_INSERT_BUFFER = load_solution_dfs(
        data_path,
        instance,
        dist_method,
        algorithm='INSERT_BUFFER',
        include_all_city=include_all_city
    )
    algorithms.append({
        "name": "INSERT_BUFFER",
        "labors_df": labors_algo_INSERT_BUFFER_df,
        "moves_df": moves_algo_INSERT_BUFFER_df,
        "type": "algorithm",
        "color": "#16A085",
        "visible": True,
    })

if algo_selection['REACT']:
    labors_algo_REACT_df, moves_algo_REACT_df, postponed_labors_REACT_BUFFER_0 = load_solution_dfs(
        data_path, 
        instance, 
        dist_method,
        algorithm='REACT_BUFFER_0',
        include_all_city=include_all_city
    )
    algorithms.append({
        "name": "REACT",
        "labors_df": labors_algo_REACT_df,
        "moves_df": moves_algo_REACT_df,
        "type": "algorithm",
        "color": "#E67E22",  # 🟠 Bold orange – “fast adaptation”
        "visible": True,
    })

if algo_selection['REACT_BUFFER']:
    labors_algo_REACT_BUFFER_df, moves_algo_REACT_BUFFER_df, postponed_labors_REACT_BUFFER = load_solution_dfs(
        data_path, 
        instance, 
        dist_method,
        algorithm='REACT_BUFFER',
        include_all_city=include_all_city
    )
    algorithms.append({
        "name": "REACT_BUFFER",
        "labors_df": labors_algo_REACT_BUFFER_df,
        "moves_df": moves_algo_REACT_BUFFER_df,
        "type": "algorithm",
        "color": "#9B59B6",  # 🟣 Electric purple – “intelligent batching”
        "visible": True,
    })

# Resultados

In [ ]:


def plot_metrics_comparison_interactive_dynamic(
    algorithms, cities, metricas, dist_dict, fechas, metrics_with_gap=None
):
    """Fully dynamic interactive version for any number of algorithms."""

    city_dropdown = widgets.Dropdown(
        options=['ALL'] + sorted(list(set(cities))),
        value='ALL',
        description="Ciudad:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="300px")
    )

    # dynamically create visibility checkboxes
    algo_checks = {
        algo["name"]: widgets.Checkbox(value=algo.get("visible", True), description=algo["name"])
        for algo in algorithms
    }

    def _plot(city, **vis_flags):
        algo_copy = [{**algo, "visible": vis_flags[algo["name"]]} for algo in algorithms]
        plot_metrics_comparison_dynamic(
            algo_copy,
            city=city,
            metricas=metricas,
            dist_dict=dist_dict,
            fechas=fechas,
            metrics_with_gap=metrics_with_gap,
            save_dir=None
        )

    interact(
        _plot,
        city=city_dropdown,
        **algo_checks
    )

metrics_with_gap = [
    "total_distance",
    "driver_move_distance",
    "labor_extra_time",
    "driver_extra_time"
]

plot_metrics_comparison_interactive_dynamic(
    algorithms,
    cities=valid_cities,
    metricas=metricas,
    dist_dict=dist_dict,
    fechas=(fechas[0], fechas[-1]),
    metrics_with_gap=metrics_with_gap
)


interactive(children=(Dropdown(description='Ciudad:', layout=Layout(width='300px'), options=('ALL', '1', '1004…

# Validación de soluciones

In [5]:
from src.algorithms.solution_validator import validate_all_algorithms
validation_summary, moves_issues = validate_all_algorithms(
    algorithms[1:], 
    dist_method=dist_method,
    dist_dict=dist_dict
    )
display(validation_summary)



🚀 Validating Offline ...
🧑‍🔧 Checked 41 drivers, found 0 overlaps
✅ Validated 233 drivers, 3018 rows, 0 inconsistencies

🚀 Validating INSERT ...
🧑‍🔧 Checked 42 drivers, found 114 overlaps
✅ Validated 240 drivers, 3021 rows, 0 inconsistencies

🚀 Validating INSERT_BUFFER ...
🧑‍🔧 Checked 42 drivers, found 115 overlaps
✅ Validated 232 drivers, 3021 rows, 2 inconsistencies

🚀 Validating REACT ...
🧑‍🔧 Checked 42 drivers, found 0 overlaps
✅ Validated 229 drivers, 3021 rows, 0 inconsistencies

🚀 Validating REACT_BUFFER ...
🧑‍🔧 Checked 42 drivers, found 0 overlaps
✅ Validated 232 drivers, 3015 rows, 0 inconsistencies

📊 Validation Summary:


,algorithm,n_labors,n_moves,overlaps,move_inconsistencies,drivers_checked,rows_checked,pairs_checked,distance_calls,inconsistencies
0,Offline,1180,3018,0,0,233,3018,3018,3018,0
1,INSERT,1147,3021,114,0,240,3021,3021,3021,0
2,INSERT_BUFFER,1147,3021,115,2,232,3021,3021,3021,2
3,REACT,1035,3021,0,0,229,3021,3021,3021,0
4,REACT_BUFFER,1039,3015,0,0,232,3015,3015,3015,0


# Debugging

## Individual checks

In [6]:
city = '830'
fecha = '2026-01-07'

In [7]:
drop_columns = ['labor_type', 'labor_name', 'shop', 'alfred', 'labor_price', 
                'labor_created_at', 'labor_start_date', 'labor_end_date', 
                'client_type', 'paying_customer', 'state_service', 'labor_category',
                'start_address_id', 'end_address_id', 'date', 'address_id', 'address_point']
column_order = ['service_id', 'labor_id', 'created_at', 'schedule_date', 'actual_start', 'actual_end', 'map_start_point', 'map_end_point', 'city', 'assigned_driver', 'dist_km']

## NUMBER OF LABORS

In [ ]:
filtered_labors_baseline = flexible_filter(labors_algo_baseline_df,
                city=city,
                schedule_date=fecha)

filtered_labors_static = flexible_filter(labors_algo_static_df,
                city=city,
                schedule_date=fecha)

filtered_labors_INSERT = flexible_filter(labors_algo_INSERT_df,
                city=city,
                schedule_date=fecha)

filtered_labors_INSERT_BUFFER = flexible_filter(labors_algo_INSERT_BUFFER_df,
                city=city,
                schedule_date=fecha)

filtered_labors_REACT = flexible_filter(labors_algo_REACT_df,
                city=city,
                schedule_date=fecha)

filtered_labors_REACT_BUFFER = flexible_filter(labors_algo_REACT_BUFFER_df,
                city=city,
                schedule_date=fecha)

filtered_moves_baseline = flexible_filter(moves_algo_baseline_df,
                city=city,
                schedule_date=fecha)

filtered_moves_INSERT = flexible_filter(moves_algo_INSERT_df,
                city=city,
                schedule_date=fecha)

filtered_moves_INSERT_BUFFER = flexible_filter(moves_algo_INSERT_BUFFER_df,
                city=city,
                schedule_date=fecha)

filtered_moves_REACT = flexible_filter(moves_algo_REACT_df,
                city=city,
                schedule_date=fecha)

filtered_moves_REACT_BUFFER = flexible_filter(moves_algo_REACT_BUFFER_df,
                city=city,
                schedule_date=fecha)

print(f'-------- Labors')
print(f'Baseline:      {len(filtered_labors_baseline)}')
print(f'INSERT:        {len(filtered_labors_INSERT)}')
print(f'INSERT BUFFER: {len(filtered_labors_INSERT_BUFFER)}')
print(f'REACT:         {len(filtered_labors_REACT)}')
print(f'REACT BUFFER:  {len(filtered_labors_REACT_BUFFER)}')

print(f'\n-------- Moves')
print(f'Baseline:      {len(filtered_moves_baseline)}')
print(f'INSERT:        {len(filtered_moves_INSERT)}')
print(f'INSERT BUFFER: {len(filtered_moves_INSERT_BUFFER)}')
print(f'REACT:         {len(filtered_moves_REACT)}')
print(f'REACT BUFFER:  {len(filtered_moves_REACT_BUFFER)}')

-------- Labors
Baseline:      2
INSERT:        2
INSERT BUFFER: 2
REACT:         2
REACT BUFFER:  2

-------- Moves
Baseline:      6
INSERT:        6
INSERT BUFFER: 6
REACT:         6
REACT BUFFER:  6


## EXTRA SERVICES

Apparently there are some services which are repeated (someway). This service should be cancelled completely if one of the services cannot be fulfilled.
Esto cause que INSERT E INSERTBUFFER tengan un aparente servicio adicional en 07.01.26.

In [9]:
filtered_labors_baseline

,service_id,labor_id,labor_type,labor_name,labor_category,labor_price,labor_created_at,labor_start_date,labor_end_date,alfred,...,address_point,address_name,labor_sequence,date,map_start_point,map_end_point,assigned_driver,actual_start,actual_end,dist_km
0,213400,306065,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,65004.0,2025-02-17 14:23:24.316000-05:00,2026-01-07 14:49:00-05:00,2026-01-07 15:35:00-05:00,70080,...,POINT (-75.6678067 4.799966),Casa,0,2026-01-07,POINT (-75.7362559 4.815196300000001),POINT (-75.7016694 4.811435),70080,2026-01-07 15:00:00-05:00,2026-01-07 15:50:46.953170-05:00,3.855035
1,213411,306080,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,65004.0,2025-02-17 14:42:00.422000-05:00,2026-01-07 16:39:00-05:00,2026-01-07 17:04:00-05:00,70080,...,POINT (-75.6678067 4.799966),Casa,0,2026-01-07,POINT (-75.7016694 4.811435),POINT (-75.7362559 4.815196300000001),70080,2026-01-07 16:30:00-05:00,2026-01-07 17:20:46.953170-05:00,3.855035


In [19]:
labors_algo_baseline_df.columns

Index(['service_id', 'labor_id', 'labor_type', 'labor_name', 'labor_category',
       'labor_price', 'labor_created_at', 'labor_start_date', 'labor_end_date',
       'alfred', 'shop', 'created_at', 'schedule_date', 'client_type',
       'paying_customer', 'state_service', 'start_address_id',
       'start_address_point', 'end_address_id', 'end_address_point', 'city',
       'address_id', 'address_point', 'address_name', 'labor_sequence', 'date',
       'map_start_point', 'map_end_point', 'assigned_driver', 'actual_start',
       'actual_end', 'dist_km'],
      dtype='object')

In [11]:
flexible_filter(
    labors_algo_baseline_df,
    service_id='232102'
).sort_values(['schedule_date', 'actual_start', 'actual_end'])

,service_id,labor_id,labor_type,labor_name,labor_category,labor_price,labor_created_at,labor_start_date,labor_end_date,alfred,...,address_point,address_name,labor_sequence,date,map_start_point,map_end_point,assigned_driver,actual_start,actual_end,dist_km
2,232102,327400,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,57789.0,2025-04-21 07:40:54.844000-05:00,2026-01-06 09:35:00-05:00,2026-01-06 11:38:00-05:00,72164,...,POINT (-75.7102592 4.801893499999999),Casa,4,2026-01-06,POINT (-75.7102592 4.801893499999999),POINT (-75.7102592 4.801893499999999),72164,2026-01-06 16:00:00-05:00,2026-01-06 16:45:00-05:00,0.000000
3,232102,327417,16.0,Tanqueo Terpel (Corriente),OTHER_LABORS,47600.0,2025-04-21 08:16:06.653000-05:00,2026-01-06 11:38:00-05:00,2026-01-06 12:00:00-05:00,,...,POINT (-75.6927012 4.810058499999999),Terpel - Pereira,6,2026-01-06,POINT (-75.6927012 4.810058499999999),POINT (-75.6927012 4.810058499999999),<NA>,2026-01-06 16:45:00-05:00,2026-01-06 16:49:11.178571-05:00,0.000000
4,232102,326475,2.0,Alfred Transport,VEHICLE_TRANSPORTATION,57789.0,2025-04-15 14:35:15.906000-05:00,2026-01-06 07:41:00-05:00,2026-01-06 10:03:00-05:00,72164,...,POINT (-75.7102592 4.801893499999999),Casa,2,2026-01-06,POINT (-75.7193049 4.8156245),POINT (-75.7102592 4.801893499999999),72164,2026-01-06 16:49:11.178571-05:00,2026-01-06 17:36:55.555335-05:00,1.826408
5,232102,327399,6.0,General Mechanics,GENERAL_MECHANICS,1242360.0,2025-04-21 07:40:54.796000-05:00,2026-01-06 10:03:00-05:00,2026-01-06 09:35:00-05:00,,...,POINT (-75.6770009 4.830667699999999),Serviautos Pereira,3,2026-01-06,POINT (-75.6770009 4.830667699999999),POINT (-75.6770009 4.830667699999999),<NA>,2026-01-06 17:36:55.555335-05:00,2026-01-07 09:02:03.412478-05:00,0.000000
10,232102,326473,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,57789.0,2025-04-15 14:32:57.273000-05:00,2026-01-06 16:00:00-05:00,2026-01-06 18:21:00-05:00,72164,...,POINT (-75.7102592 4.801893499999999),Casa,0,2026-01-06,POINT (-75.7102592 4.801893499999999),POINT (-75.7102592 4.801893499999999),<NA>,NaT,NaT,0.000000
9,232102,326474,4.0,Tecnomecanica,SAFETI_INSPECTION,318200.0,2025-04-15 14:32:57.325000-05:00,2026-01-06 18:21:00-05:00,2026-01-06 07:41:00-05:00,,...,POINT (-75.6613069 4.850945),CDA Cardisel SAS La Romelia - Pereira,1,2026-01-06,POINT (-75.6613069 4.850945),POINT (-75.6613069 4.850945),<NA>,NaT,NaT,0.000000
7,232102,327416,4.0,Tecnomecanica,SAFETI_INSPECTION,296128.0,2025-04-21 08:13:20.415000-05:00,2026-01-06 11:38:00-05:00,2026-01-06 14:18:00-05:00,,...,POINT (-75.6613069 4.850945),CDA Cardisel SAS La Romelia - Pereira,7,2026-01-06,POINT (-75.6613069 4.850945),POINT (-75.6613069 4.850945),<NA>,NaT,NaT,0.000000
6,232102,327418,755.0,Alfred Service Transport,VEHICLE_TRANSPORTATION,45000.0,2025-04-21 08:16:06.670000-05:00,2026-01-06 11:04:00-05:00,2026-01-06 11:38:00-05:00,72164,...,POINT (-75.7102592 4.801893499999999),Casa,5,2026-01-06,POINT (-75.6770009 4.830667699999999),POINT (-75.7102592 4.801893499999999),<NA>,NaT,NaT,0.000000
8,232102,327541,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,57789.0,2025-04-21 12:09:40.098000-05:00,2026-01-06 14:18:00-05:00,2026-01-06 14:32:00-05:00,72164,...,POINT (-75.7102592 4.801893499999999),Casa,8,2026-01-06,POINT (-75.6927012 4.810058499999999),POINT (-75.7102592 4.801893499999999),<NA>,NaT,NaT,0.000000


In [18]:
filtered_labors_INSERT

,service_id,labor_id,labor_type,labor_name,labor_category,labor_price,labor_created_at,labor_start_date,labor_end_date,alfred,...,labor_sequence,date,map_start_point,map_end_point,assigned_driver,actual_start,actual_end,dist_km,latest_arrival_time,n_drivers
1070,213400,306065,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,65004.0,2025-02-17 14:23:24.316000-05:00,2026-01-07 14:49:00-05:00,2026-01-07 15:35:00-05:00,70080,...,0,2026-01-07,NaN,NaN,70080,2026-01-07 15:00:00-05:00,2026-01-07 15:50:46.953170-05:00,3.855035,2026-01-07 15:45:00-05:00,NaN
1071,213411,306080,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,65004.0,2025-02-17 14:42:00.422000-05:00,2026-01-07 16:39:00-05:00,2026-01-07 17:04:00-05:00,70080,...,0,2026-01-07,NaN,NaN,70080,2026-01-07 16:30:00-05:00,2026-01-07 17:20:46.953170-05:00,3.855035,2026-01-07 17:15:00-05:00,NaN


In [13]:
flexible_filter(
    labors_algo_INSERT_df,
    service_id='232102'
).sort_values(['schedule_date', 'actual_start', 'actual_end'])

,service_id,labor_id,labor_type,labor_name,labor_category,labor_price,labor_created_at,labor_start_date,labor_end_date,alfred,...,labor_sequence,date,map_start_point,map_end_point,assigned_driver,actual_start,actual_end,dist_km,latest_arrival_time,n_drivers
1064,232102,326473,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,57789.0,2025-04-15 14:32:57.273000-05:00,2026-01-06 16:00:00-05:00,2026-01-06 18:21:00-05:00,72164,...,0,2026-01-06,NaN,NaN,72164,2026-01-06 16:00:00-05:00,2026-01-06 16:45:00-05:00,0.0,2026-01-06 16:45:00-05:00,NaN
1066,232102,326475,2.0,Alfred Transport,VEHICLE_TRANSPORTATION,57789.0,2025-04-15 14:35:15.906000-05:00,2026-01-06 07:41:00-05:00,2026-01-06 10:03:00-05:00,72164,...,2,2026-01-06,NaN,NaN,72164,2026-01-06 16:45:00-05:00,2026-01-06 17:30:00-05:00,0.0,2026-01-06 16:45:00-05:00,NaN
1065,232102,326474,4.0,Tecnomecanica,SAFETI_INSPECTION,318200.0,2025-04-15 14:32:57.325000-05:00,2026-01-06 18:21:00-05:00,2026-01-06 07:41:00-05:00,,...,1,2026-01-06,NaN,NaN,,2026-01-06 16:45:00-05:00,2026-01-06 18:33:30.500000-05:00,0.0,2026-01-06 16:45:00-05:00,NaN
1067,232102,327399,6.0,General Mechanics,GENERAL_MECHANICS,1242360.0,2025-04-21 07:40:54.796000-05:00,2026-01-06 10:03:00-05:00,2026-01-06 09:35:00-05:00,,...,3,2026-01-06,NaN,NaN,,2026-01-06 17:30:00-05:00,2026-01-07 12:53:15-05:00,0.0,2026-01-06 16:45:00-05:00,NaN
1068,232102,327417,16.0,Tanqueo Terpel (Corriente),OTHER_LABORS,47600.0,2025-04-21 08:16:06.653000-05:00,2026-01-06 11:38:00-05:00,2026-01-06 12:00:00-05:00,,...,6,2026-01-06,NaN,NaN,,2026-01-07 12:53:15-05:00,2026-01-07 13:00:42.750000-05:00,0.0,2026-01-06 16:45:00-05:00,NaN
1069,232102,327416,4.0,Tecnomecanica,SAFETI_INSPECTION,296128.0,2025-04-21 08:13:20.415000-05:00,2026-01-06 11:38:00-05:00,2026-01-06 14:18:00-05:00,,...,7,2026-01-06,NaN,NaN,,2026-01-07 13:00:42.750000-05:00,2026-01-07 14:49:13.250000-05:00,0.0,2026-01-06 16:45:00-05:00,NaN


In [12]:
plot_metrics_comparison_dynamic(
    algorithms,
    city=city,
    metricas=metricas,
    dist_dict=dist_dict,
    fechas=fechas,
    metrics_with_gap=metrics_with_gap
)



['274945' '275107' '275163']
['231146' '232092' '232102']
['213400' '213411']
['216642' '216714' '216754']
['211033' '211034' '211143']
['213675' '213732']
['226543' '226554' '226656']


['274945' '275107' '275163']
['231146' '232092' '232102']
['213400' '213411']
['216642' '216714' '216754']
['211033' '211034' '211143']
['213675' '213732']
['226543' '226554' '226656']


['274945' '275107' '275163']
['231146' '232092' '232102']
['232102' '213400' '213411']
['216642' '216714' '216754']
['211033' '211034' '211143']
['213675' '213732']
['226543' '226554' '226656']


['274945' '275107' '275163']
['231146' '232092' '232102']
['232102' '213400' '213411']
['216642' '216714' '216754']
['211033' '211034' '211143']
['213675' '213732']
['226543' '226554' '226656']


['274945' '275107' '275163']
['231146' '232092' '232102']
['213400' '213411']
['216642' '216714' '216754']
['211033' '211034' '211143']
['213675' '213732']
['226543' '226554' '226656']


['274945' '275107' '275163']
['231146' '23209

## OTHER TESTING!!!

In [19]:
flexible_filter(
    labors_algo_baseline_df.drop(
        columns=drop_columns
    )[column_order],
    city=city, 
    schedule_date=fecha).sort_values(['schedule_date'])

,service_id,labor_id,created_at,schedule_date,actual_start,actual_end,map_start_point,map_end_point,city,dist_km
0,218033,311188,2026-01-03 08:22:06.741000-05:00,2026-01-05 07:00:00-05:00,2026-01-05 06:30:00-05:00,2026-01-05 07:17:09.137485-05:00,POINT (-73.11255469999999 7.117752199999999),POINT (-73.1192449 7.1288176),844,1.434861
1,218109,311287,2026-01-03 12:01:47.748000-05:00,2026-01-05 09:30:00-05:00,2026-01-05 09:00:00-05:00,2026-01-05 09:59:36.286581-05:00,POINT (-73.0782802 7.0281457),POINT (-73.11350980000002 7.108425699999999),844,9.736518
2,218905,312140,2026-01-05 11:06:20.985000-05:00,2026-01-05 16:00:00-05:00,2026-01-05 15:30:00-05:00,2026-01-05 16:22:59.420799-05:00,POINT (-73.0975205 7.0649086),POINT (-73.1149525 7.109582199999998),844,5.326898
3,218944,312188,2026-01-05 13:44:35.523000-05:00,2026-01-05 18:00:00-05:00,2026-01-05 17:30:00-05:00,2026-01-05 18:29:36.286581-05:00,POINT (-73.11350980000002 7.108425699999999),POINT (-73.0782802 7.0281457),844,9.736518


In [18]:
flexible_filter(
    labors_algo_INSERT_BUFFER_df.drop(
        columns=drop_columns
    )[column_order], 
    city=city, 
    schedule_date=fecha).sort_values(['schedule_date'])

,service_id,labor_id,created_at,schedule_date,actual_start,actual_end,map_start_point,map_end_point,city,dist_km
0,218033,311188,2026-01-03 08:22:06.741000-05:00,2026-01-05 07:00:00-05:00,2026-01-05 06:30:00-05:00,2026-01-05 07:17:09.137485-05:00,POINT (-73.11255469999999 7.117752199999999),POINT (-73.1192449 7.1288176),844,1.434861
1,218109,311287,2026-01-03 12:01:47.748000-05:00,2026-01-05 09:30:00-05:00,2026-01-05 09:00:00-05:00,2026-01-05 09:59:36.286581-05:00,POINT (-73.0782802 7.0281457),POINT (-73.11350980000002 7.108425699999999),844,9.736518


In [20]:
flexible_filter(
    labors_algo_REACT_df.drop(
        columns=drop_columns
    )[column_order], 
    city=city, 
    schedule_date=fecha).sort_values(['schedule_date'])

,service_id,labor_id,created_at,schedule_date,actual_start,actual_end,map_start_point,map_end_point,city,dist_km
0,218033,311188,2026-01-03 08:22:06.741000-05:00,2026-01-05 07:00:00-05:00,2026-01-05 06:30:00-05:00,2026-01-05 07:17:09.137485-05:00,POINT (-73.11255469999999 7.117752199999999),POINT (-73.1192449 7.1288176),844,1.434861
1,218109,311287,2026-01-03 12:01:47.748000-05:00,2026-01-05 09:30:00-05:00,2026-01-05 09:00:00-05:00,2026-01-05 09:59:36.286581-05:00,POINT (-73.0782802 7.0281457),POINT (-73.11350980000002 7.108425699999999),844,9.736518
2,218905,312140,2026-01-05 11:06:20.985000-05:00,2026-01-05 16:00:00-05:00,2026-01-05 15:30:00-05:00,2026-01-05 16:22:59.420799-05:00,POINT (-73.0975205 7.0649086),POINT (-73.1149525 7.109582199999998),844,5.326898
3,218944,312188,2026-01-05 13:44:35.523000-05:00,2026-01-05 18:00:00-05:00,2026-01-05 17:30:00-05:00,2026-01-05 18:29:36.286581-05:00,POINT (-73.11350980000002 7.108425699999999),POINT (-73.0782802 7.0281457),844,9.736518


### Moves

In [24]:
flexible_filter(
    moves_algo_baseline_df, 
    city=city, 
    schedule_date=fecha).sort_values(['schedule_date'])


,service_id,labor_id,labor_context_id,labor_name,labor_category,assigned_driver,schedule_date,actual_start,actual_end,start_point,end_point,distance_km,duration_min,city,date
0,218033,311188,311188_free,FREE_TIME,FREE_TIME,6412,2026-01-05 07:00:00-05:00,2026-01-05 06:23:45.091679-05:00,2026-01-05 06:23:45.091679-05:00,POINT (-73.1037471 7.0910492),POINT (-73.1037471 7.0910492),0.000000,0.0,844,2026-01-05
1,218033,311188,311188_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-05 07:00:00-05:00,2026-01-05 06:23:45.091679-05:00,2026-01-05 06:30:00-05:00,POINT (-73.1037471 7.0910492),POINT (-73.11255469999999 7.117752199999999),3.124236,6.2,844,2026-01-05
2,218033,311188,311188_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,6412,2026-01-05 07:00:00-05:00,2026-01-05 06:30:00-05:00,2026-01-05 07:17:09.137485-05:00,POINT (-73.11255469999999 7.117752199999999),POINT (-73.1192449 7.1288176),NaN,47.2,844,2026-01-05
3,218109,311287,311287_free,FREE_TIME,FREE_TIME,6412,2026-01-05 09:30:00-05:00,2026-01-05 07:17:09.137485-05:00,2026-01-05 08:35:51.307394-05:00,POINT (-73.1192449 7.1288176),POINT (-73.1192449 7.1288176),0.000000,78.7,844,2026-01-05
4,218109,311287,311287_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-05 09:30:00-05:00,2026-01-05 08:35:51.307394-05:00,2026-01-05 09:00:00-05:00,POINT (-73.1192449 7.1288176),POINT (-73.0782802 7.0281457),12.072438,24.1,844,2026-01-05
5,218109,311287,311287_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,6412,2026-01-05 09:30:00-05:00,2026-01-05 09:00:00-05:00,2026-01-05 09:59:36.286581-05:00,POINT (-73.0782802 7.0281457),POINT (-73.11350980000002 7.108425699999999),NaN,59.6,844,2026-01-05
6,218905,312140,312140_free,FREE_TIME,FREE_TIME,6412,2026-01-05 16:00:00-05:00,2026-01-05 09:59:36.286581-05:00,2026-01-05 15:19:41.939577-05:00,POINT (-73.11350980000002 7.108425699999999),POINT (-73.11350980000002 7.108425699999999),0.000000,320.1,844,2026-01-05
7,218905,312140,312140_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-05 16:00:00-05:00,2026-01-05 15:19:41.939577-05:00,2026-01-05 15:30:00-05:00,POINT (-73.11350980000002 7.108425699999999),POINT (-73.0975205 7.0649086),5.150504,10.3,844,2026-01-05
8,218905,312140,312140_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,6412,2026-01-05 16:00:00-05:00,2026-01-05 15:30:00-05:00,2026-01-05 16:22:59.420799-05:00,POINT (-73.0975205 7.0649086),POINT (-73.1149525 7.109582199999998),NaN,53.0,844,2026-01-05
9,218944,312188,312188_free,FREE_TIME,FREE_TIME,6412,2026-01-05 18:00:00-05:00,2026-01-05 16:22:59.420799-05:00,2026-01-05 17:29:35.443094-05:00,POINT (-73.1149525 7.109582199999998),POINT (-73.1149525 7.109582199999998),0.000000,66.6,844,2026-01-05


In [23]:
flexible_filter(
    moves_algo_REACT_df, 
    city=city, 
    schedule_date=fecha).sort_values(['schedule_date'])

,service_id,labor_id,labor_context_id,labor_name,labor_category,assigned_driver,schedule_date,actual_start,actual_end,start_point,end_point,distance_km,duration_min,city,date
0,218033,311188,311188_free,FREE_TIME,FREE_TIME,6412,2026-01-05 07:00:00-05:00,2026-01-05 06:23:45.091679-05:00,2026-01-05 06:23:45.091679-05:00,POINT (-73.1037471 7.0910492),POINT (-73.1037471 7.0910492),0.000000,0.0,844,2026-01-05
1,218033,311188,311188_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-05 07:00:00-05:00,2026-01-05 06:23:45.091679-05:00,2026-01-05 06:30:00-05:00,POINT (-73.1037471 7.0910492),POINT (-73.11255469999999 7.117752199999999),3.124236,6.2,844,2026-01-05
2,218033,311188,311188_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,6412,2026-01-05 07:00:00-05:00,2026-01-05 06:30:00-05:00,2026-01-05 07:17:09.137485-05:00,POINT (-73.11255469999999 7.117752199999999),POINT (-73.1192449 7.1288176),NaN,47.2,844,2026-01-05
3,218109,311287,311287_free,FREE_TIME,FREE_TIME,6412,2026-01-05 09:30:00-05:00,2026-01-05 07:17:09.137485-05:00,2026-01-05 08:35:51.307394-05:00,POINT (-73.1192449 7.1288176),POINT (-73.1192449 7.1288176),0.000000,78.7,844,2026-01-05
4,218109,311287,311287_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-05 09:30:00-05:00,2026-01-05 08:35:51.307394-05:00,2026-01-05 09:00:00-05:00,POINT (-73.1192449 7.1288176),POINT (-73.0782802 7.0281457),12.072438,24.1,844,2026-01-05
5,218109,311287,311287_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,6412,2026-01-05 09:30:00-05:00,2026-01-05 09:00:00-05:00,2026-01-05 09:59:36.286581-05:00,POINT (-73.0782802 7.0281457),POINT (-73.11350980000002 7.108425699999999),NaN,59.6,844,2026-01-05


In [31]:
df = moves_algo_baseline_df
df = df[df['city'] != 'ALL']
df

,service_id,labor_id,labor_context_id,labor_name,labor_category,assigned_driver,schedule_date,actual_start,actual_end,start_point,end_point,distance_km,duration_min,city,date
0,205621,297525,297525_free,FREE_TIME,FREE_TIME,10491,2026-01-05 07:30:00-05:00,2026-01-05 06:53:04.404846-05:00,2026-01-05 06:53:04.404846-05:00,POINT (-75.5948708 6.222133500000001),POINT (-75.5948708 6.222133500000001),0.000000,0.0,1,2026-01-05
1,205621,297525,297525_move,DRIVER_MOVE,DRIVER_MOVE,10491,2026-01-05 07:30:00-05:00,2026-01-05 06:53:04.404846-05:00,2026-01-05 07:00:00-05:00,POINT (-75.5948708 6.222133500000001),POINT (-75.5722612 6.2005726),3.463293,6.9,1,2026-01-05
2,205621,297525,297525_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,10491,2026-01-05 07:30:00-05:00,2026-01-05 07:00:00-05:00,2026-01-05 07:49:31.984874-05:00,POINT (-75.5722612 6.2005726),POINT (-75.56991479999999 6.2276503),NaN,49.5,1,2026-01-05
3,206697,298694,298694_free,FREE_TIME,FREE_TIME,31857,2026-01-05 08:00:00-05:00,2026-01-05 07:23:59.172324-05:00,2026-01-05 07:23:59.172324-05:00,POINT (-75.58290559999999 6.155980599999999),POINT (-75.58290559999999 6.155980599999999),0.000000,0.0,1,2026-01-05
4,206697,298694,298694_move,DRIVER_MOVE,DRIVER_MOVE,31857,2026-01-05 08:00:00-05:00,2026-01-05 07:23:59.172324-05:00,2026-01-05 07:30:00-05:00,POINT (-75.58290559999999 6.155980599999999),POINT (-75.5673676 6.1781754),3.006897,6.0,1,2026-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,215474,308380,308380_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-11 16:30:00-05:00,2026-01-11 15:55:44.773032-05:00,2026-01-11 16:00:00-05:00,POINT (-73.1037471 7.0910492),POINT (-73.11345413684005 7.107574257887976),2.126891,4.3,844,2026-01-11
11,215474,308380,308380_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,6412,2026-01-11 16:30:00-05:00,2026-01-11 16:00:00-05:00,2026-01-11 16:46:46.377629-05:00,POINT (-73.11345413684005 7.107574257887976),POINT (-73.1063319 7.1155142),NaN,46.8,844,2026-01-11
6,215495,308403,308403_free,FREE_TIME,FREE_TIME,6412,2026-01-11 13:00:00-05:00,2026-01-11 10:19:15.875810-05:00,2026-01-11 12:14:23.509259-05:00,POINT (-73.1134749 7.1084515),POINT (-73.1134749 7.1084515),0.000000,115.1,844,2026-01-11
7,215495,308403,308403_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-11 13:00:00-05:00,2026-01-11 12:14:23.509259-05:00,2026-01-11 12:30:00-05:00,POINT (-73.1134749 7.1084515),POINT (-73.181381 7.128081700000001),7.804090,15.6,844,2026-01-11


In [33]:
df = moves_algo_REACT_df
df = df[df['city'] != 'ALL']
df[df['service_id'] == '218944']

,service_id,labor_id,labor_context_id,labor_name,labor_category,assigned_driver,schedule_date,actual_start,actual_end,start_point,end_point,distance_km,duration_min,city,date
9,218944,312188,312188_free,FREE_TIME,FREE_TIME,6412,2026-01-05 18:00:00-05:00,2026-01-05 16:22:59.420799-05:00,2026-01-05 17:29:35.443094-05:00,POINT (-73.1149525 7.109582199999998),POINT (-73.1149525 7.109582199999998),0.000000,66.6,nan,NaN
10,218944,312188,312188_move,DRIVER_MOVE,DRIVER_MOVE,6412,2026-01-05 18:00:00-05:00,2026-01-05 17:29:35.443094-05:00,2026-01-05 17:30:00-05:00,POINT (-73.1149525 7.109582199999998),POINT (-73.11350980000002 7.108425699999999),0.204641,0.4,nan,NaN
11,218944,312188,312188_labor,Alfred Initial Transport,VEHICLE_TRANSPORTATION,6412,2026-01-05 18:00:00-05:00,2026-01-05 17:30:00-05:00,2026-01-05 18:29:36.286581-05:00,POINT (-73.11350980000002 7.108425699999999),POINT (-73.0782802 7.0281457),NaN,59.6,nan,NaN
